In [382]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import os
from operator import itemgetter, attrgetter, methodcaller
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

# Instead of training models for each note
- decide which will have notes, for now pick x with one, y with another (train model for this + holds, hands, mines, rolls etc later)
- for each note that will have something, decide what combo it has (train from prev notes (not all 48, pick more relevant ones)) (4 for one note, 6 for 2, 4 for 3) and pick highest class

### TODOS
- try to predict hold/roll there (would need to train on later beat info as well)
- generate percent of single double notes etc with a nn

In [338]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.note_classes = pd.read_csv('generated_data/{0}_note_classes_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [339]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_note_classes_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [356]:
beats_to_track = 48
num_classes_one_note = 4
num_classes_two_note = 6
class_map_one_note = {
    '1000': 0,
    '0100': 1,
    '0010': 2,
    '0001': 3
}
class_reverse_map_one_note = ['1000', '0100', '0010', '0001']

class_map_two_note = {
    '1001': 0,
    '0110': 1,
    '1100': 2,
    '1010': 3,
    '0101': 4,
    '0011': 5
}
class_reverse_map_two_note = ['1001', '0110', '1100', '1010', '0101', '0011']

note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

important_indices = [1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
def get_features(index, features, note_classes):
    indices = [index + song_padding - i for i in important_indices]
    return np.array([np.concatenate((note_classes[i], features[i]), axis = 0) for i in indices]).flatten()

In [365]:
song_padding = beats_to_track * 2
X_one_note = []
y_one_note = []
X_two_note = []
y_two_note = []
for key in songs:
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), songs[key].note_classes), axis = 0)
    notes = np.concatenate((([['0000']] * song_padding), songs[key].notes), axis = 0)
    if abs(len(note_classes) - len(notes) > 250):
        print ('Lengths dont match for {0}'.format(key))
        print ('{0} vs {1}'.format(len(note_classes), len(notes)))
        continue
    length = min(len(note_classes), len(notes)) - song_padding
    features = np.array([get_features_for_row(notes[i][0]) for i in range(-song_padding, length)])
    for i in range(length):
        row = notes[i + song_padding][0]
        (blank, steps, mines, hold_starts, roll_starts, hold_ends) = [row.count(note_type) for note_type in note_types]
        if steps == 1 and blank == 3:
            X_one_note.append(get_features(i, features, note_classes))
            y_one_note.append([int(char == '1') for char in row])
            
        if steps == 2 and blank == 2:
            X_two_note.append(get_features(i, features, note_classes))
            y_two_note.append([int(char == '1') for char in row])

X_one_note = np.array(X_one_note)
y_one_note = np.array(y_one_note)
X_two_note = np.array(X_two_note)
y_two_note = np.array(y_two_note)

In [360]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(100, input_dim=713, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

In [361]:
#y_one_note_hot = np.zeros((len(y_one_note), num_classes_one_note))
#y_one_note_hot[np.arange(len(y_one_note)), y_one_note] = 1
#y_two_note_hot = np.zeros((len(y_two_note), num_classes_two_note))
#y_two_note_hot[np.arange(len(y_two_note)), y_two_note] = 1

In [366]:
models = []
for i in range(7):
    model = 
    
    models.append(model)
y_one_note_model = build_model(4)
y_one_note_model.fit(X_one_note, y_one_note, nb_epoch=1, batch_size=50)
y_one_note_model.save('models/y_one_note_model.h5')
# Dense 1000: 63s - loss: 1.3934 - acc: 0.2707
# Dense 550: 53s - loss: 1.3903 - acc: 0.2703
# Dense 100: 30s - loss: 1.3950 - acc: 0.2699

Epoch 1/1
73149/73149 [==============================] - 30s - loss: 1.3946 - acc: 0.2653    


In [368]:
y_two_note_model = build_model(4)
y_two_note_model.fit(X_two_note, y_two_note, nb_epoch=1, batch_size=5)
y_two_note_model.save('models/y_two_note_model.h5')

Epoch 1/1
5160/5160 [==============================] - 14s - loss: 2.9074 - acc: 0.2479    


In [ ]:
X = [[]] * 7
y = [[]] * 7
for key in songs:
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), songs[key].note_classes), axis = 0)
    notes = np.concatenate((([['0000']] * song_padding), songs[key].notes), axis = 0)
    if abs(len(note_classes) - len(notes) > 250):
        print ('Lengths dont match for {0}'.format(key))
        print ('{0} vs {1}'.format(len(note_classes), len(notes)))
        continue
    length = min(len(note_classes), len(notes)) - song_padding
    features = np.array([get_features_for_row(notes[i][0]) for i in range(-song_padding, length)])
    for i in range(length):
        row = notes[i + song_padding][0]
        (blank, steps, mines, hold_starts, roll_starts, hold_ends) = [row.count(note_type) for note_type in note_types]
        steps += hold_starts + roll_starts
        if steps == 1:
            X[1].append(get_features(i, features, note_classes))
            y[1].append([int(char == '1' or char == '2' or char == '4') for char in row])
            
        if steps == 2:
            X[2].append(get_features(i, features, note_classes))
            y[2].append([int(char == '1' or char == '2' or char == '4') for char in row])

X = [np.array(X_for_class) for X_for_class in X]
y = [np.array(y_for_class) for y_for_class in y]

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [449]:
def get_average_for_class(note_classes, i, class_num):
    surrounding_beats = note_classes[max(i - 24,0):min(i + 24,len(note_classes))]
    return sum([beat[class_num] for beat in surrounding_beats]) / float(len(surrounding_beats))

def normalize_row(note_classes, i):
    return [note_classes[i][class_num] - get_average_for_class(note_classes, i, class_num) for class_num in range(7)]
    
def normalize_classes(note_classes):
    return [normalize_row(note_classes, i) for i in range(len(note_classes))]

In [450]:
pattern = ['1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010', '1000', '0100', '0001', '0010', '0100', '1000', '0001', '0010']
cutoff_per_class = [0, 0.9, 0.97, 0.99, 0.99, 0.99, 0.99]
def get_output(song):
    predicted_notes = []
    # maybe normalize with surroundings here?
    normalized_note_classes = normalize_classes(song.note_classes)
    sortedLists = [sorted(normalized_note_classes, key=itemgetter(i)) for i in range(7)]
    num_samples = len(song.note_classes)
    cutoffs = [sortedLists[i][int(num_samples * cutoff_per_class[i])][i] for i in range(7)]
    
    note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), song.note_classes), axis = 0)
    dummy_rows = [row for eigth in pattern for row in [eigth] + ['0000'] * 5]
    features = [get_features_for_row(row) for row in dummy_rows]
    for i in range(num_samples):
        note_class = song.note_classes[i]
        normalized_note_class = normalized_note_classes[i]
        if normalized_note_class[2] > cutoffs[2]:
            X_row = get_features(len(features) - song_padding, features, note_classes)
            prediction_values = y_one_note_model.predict(np.array([X_row]))[0]
            cutoff = sorted(prediction_values)[2]
            prediction = ''.join(['1' if value > cutoff else '0' for value in prediction_values])
        elif normalized_note_class[1] > cutoffs[1]:
            X_row = get_features(len(features) - song_padding, features, note_classes)
            prediction_values = y_two_note_model.predict(np.array([X_row]))[0]
            cutoff = sorted(prediction_values)[1]
            prediction = ''.join(['1' if value > cutoff else '0' for value in prediction_values])
        else:
            prediction = '0000'

        predicted_notes.append(prediction)
        features.append(get_features_for_row(prediction))
    return predicted_notes

In [452]:
song = songs['In The Groove~Lemmings on the Run']
song = songs['In The Groove~Infection']
song = songs['In The Groove~July']
song.predicted_notes = get_output(song)
step_song(song)

In [453]:
a = song.note_classes

In [457]:
b = np.array(normalize_classes(song.note_classes))

In [459]:
a[:10]

array([[  9.49594498e-01,   4.20812555e-02,   1.93656201e-03,
          1.01939877e-04,   1.71089510e-03,   5.52011188e-04,
          4.02283063e-03],
       [  9.86023903e-01,   1.12156728e-02,   4.54962690e-04,
          8.43358721e-05,   5.76494320e-04,   3.73267219e-04,
          1.27134717e-03],
       [  9.84605849e-01,   1.24302963e-02,   4.99614573e-04,
          8.56916522e-05,   6.21342391e-04,   3.92166578e-04,
          1.36504218e-03],
       [  9.85516250e-01,   1.17485840e-02,   4.60565585e-04,
          8.27549957e-05,   5.41534799e-04,   3.80951155e-04,
          1.26936077e-03],
       [  9.88388181e-01,   9.48203076e-03,   3.45575361e-04,
          7.55064102e-05,   3.89814028e-04,   3.39633552e-04,
          9.79245058e-04],
       [  9.89977837e-01,   8.31787847e-03,   2.80883833e-04,
          6.77514472e-05,   2.76932667e-04,   3.00909858e-04,
          7.77790148e-04],
       [  9.80041146e-01,   1.69166755e-02,   7.35904381e-04,
          7.88201869e-05,   6.76

In [460]:
b[:10]

array([[  1.07613428e-01,  -8.66409174e-02,  -1.27340258e-02,
          1.62010714e-05,  -5.36129719e-03,   9.64751307e-05,
         -2.98987387e-03],
       [  1.68924848e-01,  -1.36783520e-01,  -1.75165897e-02,
         -2.86746945e-07,  -7.98450068e-03,  -8.31096910e-05,
         -6.55686011e-03],
       [  1.61850975e-01,  -1.31029217e-01,  -1.68477304e-02,
          8.20221780e-07,  -7.64860285e-03,  -6.48998696e-05,
         -6.26134470e-03],
       [  1.58645172e-01,  -1.28465006e-01,  -1.63773972e-02,
         -2.70635765e-06,  -7.49024973e-03,  -8.05391982e-05,
         -6.22927482e-03],
       [  1.65973803e-01,  -1.34807654e-01,  -1.66597359e-02,
         -1.06079860e-05,  -7.66113307e-03,  -1.30806713e-04,
         -6.70388216e-03],
       [  1.66228314e-01,  -1.35081672e-01,  -1.64335336e-02,
         -1.90831516e-05,  -7.62575129e-03,  -1.76499592e-04,
         -6.89179340e-03],
       [  1.51458317e-01,  -1.22573395e-01,  -1.54704554e-02,
         -8.08119051e-06,  -6.99

In [ ]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()